In [2]:
#######################################################################
###### HIPPOCAMPAL INTEGRATE-AND-FIRE WITH INT INHIBITION MODEL  ######
#######################################################################
# MV-Buzsaki Lab 2020, based on Valero et al, 2020

# Loading packages
from brian2 import *
# %matplotlib inline
# from brian2tools import *
import numpy, random, warnings, time, os
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import pandas as pd
prefs.codegen.target = 'cython'  # use the Python fallback
clear_cache('cython')

In [3]:
start_scope()

## Model parameters
saveResults = True
N = 5000                 # Network size
dur = 10                  # simulation duration before interneurons boost, in seconds, prev 10
durPost = 25              # simulation duration after interneurons boost, in second, prev 25
Perc_inh = .2            # percentage of inhibitory neurons
El_E = -55.8*mV          # Leak potential E -55.53*mV 
El_I = -55.6*mV          # Leak potential I
delta_CNO_El = .5         # Change of leak potential after CNO injection simulation (in Volts)
time_transition = 5     # Time to reach the leak potential target after CNO injection (in seconds), 5
dtime_tran = 0.02*second # delta de time transition (in seconds)
perc_dreadds = .7        # percentaje of cells expressing DREADDs
sigma = 4*mV             # Gaussian white noise std dev, prev 3
dsigma = 1.5             # sigma factor dispersion across cells, 2

## Connection probs
prob_ee = .05            # Connection probability E to E
prob_ei = .1             # Connection probability E to I
prob_ii = .2             # Connection probability I to I
prob_ie = .3             # Connection probability I to E

## Intrinsic parameters 
# for E
taum_e = 24*ms           # membrane time constant
E_e = -52*mV             # reset spikes
Vt_e = -47*mV            # spike threshold

# for I
taum_i = 10*ms           # membrane time constant
E_i = -51*mV             # reset spikes -49.9*mV  
Vt_i = -46.6*mV            # spike threshold 

## Synapsis parameters
we_e  = 1.76 *mV          # E to E synaptic weight (voltage), Jercog 1.4, prev 1.87, 1.79
wi_e  = -.8  *mV          # I to E synaptic weight, Jercog -.35, prevoius -.3
we_i  = 1.0  *mV          # E to I synaptic weight, Jercog 5, previous 3.5
wi_i  = -1 *mV            # I to I synaptic weight, Jercog -1
tau_e = 8.0 *ms          # excitation time constant
tau_i = 2.0 *ms          # inhibition time constant

# Poisson kiks parameters
k_rate = 1
t_k = np.arange(0,dur + durPost,.02)                # Generate t
kiks_t = zeros(len(t_k))                  # Generate kiks(t)
kiks_t[random.sample(range(1,len(t_k)),int(dur/(1/k_rate)))] = 0  # 45 , previous 28
kiks = TimedArray(kiks_t*mV, 20*msecond)
ud = np.array([])

## Internal variables
N_E = int(N - (N * Perc_inh))                                  # Size of excitatory population
N_Et = int(N_E * .1)
N_I = int(N * Perc_inh)                                        # Size of inh population
N_Id = int(perc_dreadds*N_I)                                   # Size of inh population expressing DREADDs
CNO_fact = 2*delta_CNO_El/(time_transition/dtime_tran.item())  # Voltage increment factor during transition
target_CNO_El = (delta_CNO_El * mV + El_I)               # Voltage target after CNO delta_CNO_El/1000 + El_I.item()

(delta_CNO_El * mV + El_I)
## Equation parameters
# for E
eqs_E = ''' 
dv/dt   = (ge+gi-(v-El)+ sigmaFac*sqrt(tau)*xi + ds*kiks(t)*int(rand() <= .1))/tau : volt (unless refractory)
dge/dt  = -ge/tau_e : volt
dgi/dt  = -gi/tau_i : volt
tau : second
ds : 1
El : volt
sigmaFac: volt
'''

# for I
eqs_I = ''' 
dv/dt   = (ge+gi-(v-El)+ sigmaFac*sqrt(tau)*xi + ds*kiks(t)*int(rand() <= .09))/tau : volt (unless refractory)
dge/dt  = -ge/tau_e : volt
dgi/dt  = -gi/tau_i : volt
tau : second
ds : 1
El : volt
sigmaFac : volt
'''

#@network_operation(dt=40*ms) int(perc_dreadds*N_I)
#def detect_ud():
#    P_E.ds = 0.2 if mean(P_E.v) > -0.044 else 1 # 54
#    P_I.ds = 0.2 if mean(P_E.v) > -0.044 else 1
@network_operation(dt=dtime_tran)
def CNO_Int(t):                            # increse leak potential of interneurons after t = dur a CNO factor every step untill target_CNO_El
    P_I.El[:N_Id] = P_I.El[:N_Id] + (rand(N_Id)*CNO_fact) * mV if (t > dur*second and mean(P_I.El[:N_Id]) < target_CNO_El) else P_I.El[:N_Id]
    
## Declare network
# E subnetwork
P_E = NeuronGroup(N_E, eqs_E, threshold='v >= Vt_e', reset='v = E_e', refractory=1*ms, method='euler')
P_E.v = E_e                               # initial resting for E
P_E.v[:N_Et] = 'E_e + 15*mV'              # model beging with a kick
P_E.tau = taum_e                          # tau for E
P_E.ds = 1
P_E.El = El_E + (rand(N_E)-.5)*.1*mV
P_E.sigmaFac = sigma + (rand(N_E)-.5) * dsigma*mV;

# I subnetwork
P_I = NeuronGroup(N_I, eqs_I, threshold='v >= Vt_i', reset='v = E_i', refractory=1*ms, method='euler')
P_I.v = E_i                               # initial resting for I
P_I.tau = taum_i                          # tau for I
P_I.ds = 0
P_I.El = El_I + (rand(N_I)-.5)*.1*mV
P_I.sigmaFac = sigma + (rand(N_I)-.5) * dsigma*mV;

## Connect network
Ce_e = Synapses(P_E, P_E, on_pre='ge += we_e') # Definition of E to E synapsis
Ce_e.connect('i != j',p=prob_ee)
Ce_e.delay = 'rand()/2*ms'

Ce_i = Synapses(P_E, P_I, on_pre='ge += we_i') # Definition of E to I synapsis
Ce_i.connect(p=prob_ei)
Ce_i.delay = 'rand()/2*ms'

Ci_i = Synapses(P_I, P_I, on_pre='gi += wi_i') # Definition of I to I synapsis
Ci_i.connect('i != j',p=prob_ii)
Ci_i.delay = 'rand()/2*ms'

Ci_e = Synapses(P_I, P_E, on_pre='gi += wi_e') # Definition of I to E synapsis
Ci_e.connect(p=prob_ie)
Ci_e.delay = 'rand()/2*ms'

# Recording set-up
spk_mon_E = SpikeMonitor(P_E)
spk_mon_I = SpikeMonitor(P_I)
state_mon_E = StateMonitor(P_E, 'v', record=True)  # record exc cells
state_mon_I = StateMonitor(P_I, 'v', record=True)  # record inh cells
state_mon_I_El = StateMonitor(P_I, 'El', record=True)  # record inh cells
state_mon_E_El = StateMonitor(P_E, 'El', record=True)  # record inh cells
pop_mon_E = PopulationRateMonitor(P_E)
pop_mon_I = PopulationRateMonitor(P_I)

tRun = time.time()
print('Performing we_e:' + str(we_e) + ', wi_e:' + str(wi_e) + ', we_i:' + str(we_i) + ', wi_i:' + str(wi_i) + '...')                
print('SI Running...')    
run(dur * second + durPost * second)
print('SI done!')
print(str(time.time() - tRun) + 'seconds elapsed...')

## Get results
rate_E = []
for ii in range(0,N_E-1):
    rate_E.append(len(np.argwhere(spk_mon_E.i==ii))/dur)
rate_I = []
for ii in range(0,N_I-1):
    rate_I.append(len(np.argwhere(spk_mon_I.i==ii))/dur)

# plot figures
fig1 = plt.figure(figsize=(16, 8))
gs = GridSpec(2, 2, height_ratios=[4, 2])
ax0 = fig1.add_subplot(gs[0])               # axis 0 all duration recording
ax2 = fig1.add_subplot(gs[2],sharex=ax0)    # axis 1 kiks
ax1 = fig1.add_subplot(gs[1])               # axis 2 1 second example duration
ax3 = fig1.add_subplot(gs[3])               # histogram

# all recording duration
ax0.plot(spk_mon_E.t/ms, spk_mon_E.i, ',r') # red are exc
ax0.plot(spk_mon_I.t/ms, spk_mon_I.i + N_E, ',b') # blue are inh
ax0.plot([dur*1000, dur*1000],[0, N_E + N_I],'k')
ax0.plot([(dur + time_transition)*1000, (dur + time_transition)*1000],[0, N_E + N_I],'k--')
ax0.set_title('we_e:' + str(we_e) + ', wi_e:' + str(wi_e) + ', we_i:' + str(we_i) + ', wi_i:' + str(wi_i))
ax0.set_ylabel('Neuron index')
# 1 sec duration
ax1.plot(pop_mon_E.t/ms, pop_mon_E.smooth_rate(window='gaussian', width=10*ms),'r') # red are exc
ax1.plot(pop_mon_I.t/ms, pop_mon_I.smooth_rate(window='gaussian', width=10*ms),'b') # blue are inh
ax1.set_title('E: ' + str(round(mean(rate_E),2)) + 'Hz; I: ' + str(round(mean(rate_I),2)) + 'Hz')
ax1.set_ylabel('Rate'), ax1.set_xlim(10,(dur + durPost)*1000)
# mV
ax2.plot(state_mon_E.t/ms, mean(state_mon_E.v.T, axis = 1), 'r')
ax2.plot(state_mon_E.t/ms, mean(state_mon_I.v.T, axis = 1), 'b')
ax2.plot(state_mon_I.t/ms, mean(state_mon_I_El.El.T, axis = 1), 'c')
ax2.plot(state_mon_I.t/ms, mean(state_mon_E_El.El.T, axis = 1), 'm')
ax2.set_xlabel('Time [ms]'), ax2.set_ylabel('mV')
# 
ax3.hist(rate_I, bins=np.logspace(np.log10(0.1),np.log10(100), 50), facecolor='b')  # arguments are passed to np.histogram
ax3.hist(rate_E, bins=np.logspace(np.log10(0.1),np.log10(100), 50), facecolor='r')  # arguments are passed to np.histogram
ax3.set_xscale('log'), ax3.set_xlabel('Rate [Hz]'), ax3.set_ylabel('#')

## Save recording
if saveResults:
    print('Saving results...')
    sim_name = 'SISim ' + time.asctime() # Simulation name 
    sim_name = sim_name.replace(' ','_')
    sim_name = sim_name.replace(':','_')
    os.mkdir(sim_name) # creating simulation folder

    fig1.savefig(sim_name + '/summary.png')   # save the figure to file

    # Spikes and mp for E
    spk_E = pd.DataFrame([list(spk_mon_E.i), list(spk_mon_E.t/ms)])                 # saving E spikes
    spk_E.to_csv(sim_name + '/spk_E.csv', sep=',', encoding='utf-8', index=False, header=False)
    # mv_E = pd.DataFrame(column_stack((transpose(state_mon_E.v/mV), state_mon_E.t))) # saving E membrane potential
    # mv_E.to_csv(sim_name + '/mv_E.cvs', sep=',', encoding='utf-8', index=False, header=False)

    # Spikes for I
    spk_I = pd.DataFrame([list(spk_mon_I.i), list(spk_mon_I.t/ms)])                 # saving E spikes
    spk_I.to_csv(sim_name + '/spk_I.csv', sep=',', encoding='utf-8', index=False, header=False)
    # mv_I = pd.DataFrame(column_stack((transpose(state_mon_I.v/mV), state_mon_I.t))) # saving E membrane potential
    # mv_I.to_csv(sim_name + '/mv_I.cvs', sep=',', encoding='utf-8', index=False, header=False)

    # Save connectivity
    CE_E = pd.DataFrame([list(Ce_e.i), list(Ce_e.j)]) # E to E matrix connection
    CE_E.to_csv(sim_name + '/CE_E.csv', sep=',', encoding='utf-8', index=False, header=False)

    CE_I = pd.DataFrame([list(Ce_i.i), list(Ce_i.j)]) # E to I matrix connection
    CE_I.to_csv(sim_name + '/CE_I.csv', sep=',', encoding='utf-8', index=False, header=False)

    CI_E = pd.DataFrame([list(Ci_e.i), list(Ci_e.j)]) # I to E matrix connection
    CI_E.to_csv(sim_name + '/CI_E.csv', sep=',', encoding='utf-8', index=False, header=False)

    CI_I = pd.DataFrame([list(Ci_i.i), list(Ci_i.j)]) # I to I matrix connection
    CI_I.to_csv(sim_name + '/CI_I.csv', sep=',', encoding='utf-8', index=False, header=False)

    # Save lfp
    lfp_s = mean(concatenate((transpose(state_mon_E.v/mV),transpose(state_mon_I.v/mV)), axis = 1), axis = 1)
    lfp_s = lfp_s - mean(lfp_s) 
    t_s = state_mon_I.t
    lfp = pd.DataFrame(column_stack((lfp_s, t_s))) # saving E membrane potential
    lfp.to_csv(sim_name + '/lfp.csv', sep=',', encoding='utf-8', index=False, header=False)

    # Save features
    feat = pd.DataFrame([N_E,N_I,Perc_inh,dur,durPost,El_E.item()*1000,El_I.item()*1000,sigma.item()*1000,prob_ee,prob_ei,prob_ii,prob_ie,
                     taum_e.item()*1000,E_e.item()*1000,Vt_e.item()*1000,taum_i.item()*1000,
                     E_i.item()*1000,Vt_i.item()*1000,
                     we_e.item()*1000,wi_e.item()*1000,we_i.item()*1000,wi_i.item()*1000,tau_e.item()*1000,tau_i.item()*1000,
                     k_rate,delta_CNO_El,time_transition,perc_dreadds,dsigma])
    feat.to_csv(sim_name + '/feat.csv', sep=',', encoding='utf-8', index=False, header=False)

print('Done!')    

Performing we_e:1.76 mV, wi_e:-0.8 mV, we_i:1. mV, wi_i:-1. mV...
SI Running...


DimensionMismatchError: Cannot calculate [-55.58529223 -55.63342551 ... -55.57498067 -55.55500146] mV + [3.41317892 0.32532496 ... 3.85696682 2.73319867] uWb, units do not match (units are V and Wb).